 # Iterative CNN Approach with optimized thresholds
 
I use the great [kernel](https://www.kaggle.com/yakuben/crgl2020-iterative-cnn-approach) of Yakubenko Oleksii for model architecture and training.

Instead of setting a common threshold of y_pred = (y_pred_continuous > 0.5) for binarizing the predictions,
I determine the threshold for each board individually:

- Go through a set of possible thresholds `[0, 0.1, ..., 1]`.
- Binarize the board according to the threshold, i.e. `y_pred = (y_pred_continuous > threshold)`.
- Make N steps `y_pred_N = make_move(y_pred, N)` and compute `accuracy(y_pred_N, y_true_N)`. Note that `y_true_N` is the input to the neural network, i.e. the state of the game at time `N`.
- Use the threshold which maximizes the accuracy.
 
 
 
 
 
 This kernel uses the cythonized make_board method from [here](https://www.kaggle.com/ptyshevs/cnn-for-reversing-game-of-life)
 and [here](https://www.kaggle.com/jpmiller/demo-cython-generator-and-keras-cnn).
 
 

# Task overview/Game Rules

*The game consists of a board of cells that are either on or off. One creates an initial configuration of these on/off states and observes how it evolves. There are four simple rules to determine the next state of the game board, given the current state:*

* #### Overpopulation: if a living cell is surrounded by more than three living cells, it dies.
* #### Stasis: if a living cell is surrounded by two or three living cells, it survives.
* #### Underpopulation: if a living cell is surrounded by fewer than two living cells, it dies.
* #### Reproduction: if a dead cell is surrounded by exactly three cells, it becomes a live cell.

![](https://natureofcode.com/book/imgs/chapter07/ch07_01.png)

# Approach overview


### Iterative model
The basic idea is to train a model that can predict start state from stop state(with delta=1) and apply this model delta times.

The model itself can be any (not only neural networks).


![](https://i.ibb.co/y0fMSCg/simple-model-diagram.png)


### Iterative model with Encoder/Decoder

Using an encoder and a decoder allows more information to be conveyed to the next step (not just one channel).

![](https://i.ibb.co/MRvZbYj/model-diagram.png)


### Iterative CNN model with Encoder/Decoder

The stop and start state can be viewed as single-channel images(so we can use CV approaches)

Input - 1 channel image

Encoder in - 1 channel image | out - N channel image

ReverseOneIterationModel - inp N channel image | out - N channel image

Decoder in - N channel image | out - 1 channel image

# Model

In [ ]:
import torch
import torch.nn as nn


class OneIterationReverseNet(nn.Module):
    def __init__(self, info_ch, ch):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(info_ch, ch, 5, padding=4, padding_mode='circular')
        self.conv2 = nn.Conv2d(ch, ch, 3, )
        self.conv3 = nn.Conv2d(ch, info_ch, 3)
        
        
    def forward(self, inp):
        x = self.relu(self.conv1(inp))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        return x
      
        
class ReverseModel(nn.Module):
    def __init__(self, info_ch=64, ch=128):
        super().__init__()
        self.relu = nn.ReLU()
        self.encoder = nn.Conv2d(1, info_ch, 7, padding=3, padding_mode='circular')# you can use other model
        self.reverse_one_iter = OneIterationReverseNet(info_ch, ch)# you can use other model
        self.decoder = nn.Conv2d(info_ch, 1, 3, padding=1, padding_mode='circular')# you can use other model
        
    
    def forward(self, stop, delta):
        x = self.relu(self.encoder(stop-0.5))
        
        for i in range(delta.max().item()):
            y = self.reverse_one_iter(x)
            
            # this 2 lines allow use samples with different delta in one batch
            mask = (delta > i).reshape(-1,1,1,1)
            x = x*(~mask).float() + y*mask.float()
        
        x = self.decoder(x)
        
        return x 

# Load data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_val = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv', index_col='id')

train, val = train_test_split(train_val, test_size=0.2, shuffle=True, random_state=42, stratify=train_val['delta'])

# Dataset

In [ ]:
from torch.utils.data import DataLoader, Dataset
from torch import FloatTensor, LongTensor


def line2grid_tensor(data, device='cuda'):
    grid = data.to_numpy().reshape((data.shape[0], 1, 25, 25))
    return FloatTensor(grid).to(device)


class TaskDataset(Dataset):
    def __init__(self, data, device='cuda'):
        self.delta = LongTensor(data['delta'].to_numpy()).to(device)
        if data.shape[1] == 1251: 
            self.start = line2grid_tensor(data.iloc[:,1:626], device)
            self.stop = line2grid_tensor(data.iloc[:,626:], device)
        else:
            self.start = None
            self.stop = line2grid_tensor(data.iloc[:,1:], device)
        
    def __len__(self):
        return len(self.delta)

    def __getitem__(self, idx):
        if self.start is None:
            return {'stop': self.stop[idx], 'delta': self.delta[idx]}
        return {'start': self.start[idx], 'stop': self.stop[idx], 'delta': self.delta[idx]}

In [ ]:
dataset_train = TaskDataset(train)
dataloader_train = DataLoader(dataset_train, batch_size=128, shuffle=True)

dataset_val = TaskDataset(val)
dataloader_val = DataLoader(dataset_val, batch_size=128, shuffle=False)

dataset_test = TaskDataset(test)
dataloader_test = DataLoader(dataset_test, batch_size=128, shuffle=False)

# Train Loop

In [ ]:
from catalyst.dl import SupervisedRunner
from catalyst.dl.callbacks import CriterionCallback, EarlyStoppingCallback, AccuracyCallback
from catalyst.contrib.nn.optimizers import RAdam, Lookahead

import collections

runner = SupervisedRunner(device='cuda', input_key=['stop', 'delta'], )

loaders = {'train': dataloader_train, 'valid': dataloader_val}#collections.OrderedDict({'train': dataloader_train, 'valid': dataloader_val})

model = ReverseModel()

optimizer = Lookahead(RAdam(params=model.parameters(), lr=1e-3))

criterion = {"bce": nn.BCEWithLogitsLoss()}

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.25, patience=2)

callbacks = [
        CriterionCallback(input_key='start', prefix="loss", criterion_key="bce"),
        EarlyStoppingCallback(patience=5),
    ]

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=loaders,
    callbacks=callbacks,
    logdir="./logs",
    num_epochs=999,
    main_metric="loss",
    minimize_metric=True,
    verbose=True,
)

# Prediction / LB estimate

### Load best model

In [ ]:
best_model = ReverseModel().to('cuda')
best_model.load_state_dict(torch.load('logs/checkpoints/best.pth')['model_state_dict'])

### LB estimate

In [ ]:
%load_ext Cython

In [ ]:
%%cython
cimport cython

import numpy as np

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cdef int calc_neighs(unsigned char[:, :] field, int i, int j, int n, int k):
    cdef:
        int neighs = 0;
        int i_min = i - 1;
        int i_pl = i + 1;
        int j_min = j - 1;
        int j_pl = j + 1;
    neighs = 0
    if i_min >= 0:
        if j_min >= 0:
            neighs += field[i_min, j_min]
        neighs += field[i_min, j]
        if j_pl < k:
            neighs += field[i_min, j_pl]
    if j_min >= 0:
        neighs += field[i, j_min]
    if j_pl < k:
        neighs += field[i, j_pl]
    if i_pl < n:
        if j_min >= 0:
            neighs += field[i_pl, j_min]
        neighs += field[i_pl, j]
        if j_pl < k:
            neighs += field[i_pl, j_pl]
    return neighs

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef make_move_cython(unsigned char[:, :] field, int moves):
    cdef:
        int _, i, j, neighs;
        int n, k;
        int switch = 0;
        unsigned char[:, :] cur_field;
        unsigned char[:, :] next_field;
    cur_field = np.copy(field)
    next_field = np.zeros_like(field, 'uint8')
    n = field.shape[0]
    k = field.shape[1]
    for _ in range(moves):
        if switch == 0:
            for i in range(n):
                for j in range(k):
                    neighs = calc_neighs(cur_field, i, j, n, k)
                    if cur_field[i, j] and neighs == 2:
                        next_field[i, j] = 1
                    elif neighs == 3:
                        next_field[i, j] = 1
                    else:
                        next_field[i, j] = 0
        else:
            for i in range(n):
                for j in range(k):
                    neighs = calc_neighs(next_field, i, j, n, k)
                    if next_field[i, j] and neighs == 2:
                        cur_field[i, j] = 1
                    elif neighs == 3:
                        cur_field[i, j] = 1
                    else:
                        cur_field[i, j] = 0
        switch = (switch + 1) % 2
    return np.array(next_field if switch else cur_field)


In [ ]:
from typing import Union, List

import torch
import numpy as np



def make_move(board: Union[torch.Tensor, np.ndarray], moves: Union[int, List[int]] = 1):
    """
    Advance the game by moves steps.
    Handles:
     - batch of shape (batch_size, 1, 25, 25)
     - batch of shape (batch_size, 25, 25)
     - single board of shape (1, 25, 25)
     - single board of shape (25, 25)

    Moves can be an int or a list of ints.

    Returns the advanced game with the same shape as the input, with same dtype and device (if torch).
    """
    if isinstance(board, torch.Tensor):
        return torch.tensor(_evolve_board(board.detach().cpu().numpy(), moves),
                            dtype=board.dtype, device=board.device)

    return np.array(_evolve_board(board, moves), dtype=board.dtype)


def _evolve_board(board, moves):
    # uint8 for make_move_cython
    board = np.array(board).astype(np.uint8)

    # whole batch with one channel
    if len(board.shape) == 4:
        assert board.shape[1:] == (1, 25, 25)
        board_evolved = _move_board_3dim(board[:, 0], moves)[:, None]

    # whole batch or single board with one channel
    elif len(board.shape) == 3:
        board_evolved = _move_board_3dim(board, moves)

    # normal board
    else:
        assert board.shape == (25, 25)
        board_evolved = make_move_cython(board, moves)

    return np.array(board_evolved, dtype=np.float32)


def _move_board_3dim(board, moves):
    if board.shape == (1, 25, 25):
        board_evolved = make_move_cython(board[0], moves)[:, None]
    else:
        assert board.shape[1:] == (25, 25)
        if isinstance(moves, int):
            moves = [moves] * len(board)
        assert len(moves) == len(board)
        board_evolved = np.array([make_move_cython(b, move) for b, move in zip(board, moves)])
    return board_evolved


In [ ]:
import torch


def batch_accuracy(x, y_pred):
    return torch.sum(torch.abs(x - y_pred), dim=(1, 2, 3))


def postprocessor(y_pred, x, moves):
    """
    Find best possible threshold for each prediction.
    Roughly equivalent to:

    X = x
    Y = y
    thresholds = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
    best_thresholds = []
    for y, x, move in zip(Y, X, moves):
        best_accuracy = 0
        best_threshold = 0
        for threshold in thresholds:
            current_accuracy = accuracy(make_move(y > threshold, move), x)
            if current_accuracy > best_accuracy:
                best_accuracy = current_accuracy
                best_thresholds = threshold
        best_thresholds.append(best_threshold)

    y_processed = [y > threshold for y, threshold in zip(Y, best_thresholds)]

    :param moves: int or list of int, specifying the number of steps in game of life.
    :param y_pred: sigmoid predictions of game t in [0, 1], shape (batch_size, 1, 25, 25).
    :param x: input game (at time t + moves), shape (batch_size, 1, 25, 25)
    :return: Binarized predictions, optimized for minimal error.
    """
    thresholds = torch.linspace(0, 1, 11, device=x.device)
    errors = torch.cat([batch_accuracy(x.float(),
                                       make_move((y_pred > threshold).float(), moves)
                                       )[None]
                        for threshold in thresholds],
                       0)
    # errors.shape = (len(threhsolds), len(x))
    best_thresholds = thresholds[torch.argmin(errors, 0)][:, None, None, None]
    return (y_pred > best_thresholds).float()


In [ ]:
import numpy as np

def predict_batch(model, batch):
    model.eval()
    with torch.no_grad():
        prediction = model(batch['stop'], batch['delta'])
        prediction = torch.sigmoid(prediction).detach().cpu().numpy()
        return prediction

    
def predict_loader(model, loader):
    predict = [predict_batch(model, batch) for batch in loader]
    predict = np.concatenate(predict)
    return predict


def validate_loader(model, loader, lb_delta=None, threshold=0.5):
    prediction_val = predict_loader(best_model, loader)
    y_val = loader.dataset.start.detach().cpu().numpy()
    

    score_unoptimized = ((prediction_val > threshold) == y_val).mean(axis=(1,2,3))
    
    delta_val = loader.dataset.delta.detach().cpu().numpy()
    prediction_val = postprocessor(torch.tensor(prediction_val),
                                   x=loader.dataset.stop.cpu(),
                                   moves=delta_val).cpu().numpy()
    score = (prediction_val == y_val).mean(axis=(1,2,3))
    
    print(f'All data accuracy (global threshold): {score_unoptimized.mean()}')
    print(f'All data accuracy (optimized threshold): {score.mean()}')
        
    delta_score = {}
    for i in range(1, 6):
        delta_score[i] = score[delta_val==i].mean()#print(f'delta={i} accuracy: {score[delta_val==i].mean()}')
        print(f'delta={i} accuracy: {delta_score[i]}')
        
    if lb_delta is not None:
        lb_delta = lb_delta.value_counts(normalize=True)
        test_score = sum([lb_delta[i]*delta_score[i] for i in range(1,6)])
        print(f'VAL score         : {1-score.mean()}')
        print(f'LB  score estimate: {1-test_score}')
    
    
def make_submission(prediction, x, moves, sample_submission_path='/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv'):
    prediction = postprocessor(prediction, x, moves).numpy().astype(int).reshape(-1, 625)

    sample_submission = pd.read_csv(sample_submission_path, index_col='id')
    sample_submission.iloc[:] = prediction
    return sample_submission

In [ ]:
validate_loader(best_model, dataloader_val, test['delta'])

### submission

In [ ]:
prediction_test = predict_loader(best_model, dataloader_test)
submission = make_submission(torch.tensor(prediction_test),
                             x=dataloader_test.dataset.stop.cpu(),
                             moves=dataloader_test.dataset.delta.detach().cpu().numpy())
submission.to_csv('submission.csv')
submission.head()